In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import re
import time
import random
from tqdm import tqdm

In [140]:
import lxml.etree

In [2]:
pages = [str(i) for i in range(0, 1, 1)] # page iterations

In [3]:
page_urls = []
for page in pages:
    base_url = 'https://www.ted.com/talks?page='+page
    page_urls.append(base_url)

In [4]:
page_urls

['https://www.ted.com/talks?page=0']

In [5]:
talks = [] # also called add_links

# find href links to talks in page content under <a> tags
pbar = tqdm(total=len(page_urls), dynamic_ncols=True, colour= 'green')
for i, page in enumerate(page_urls):
    page = requests.get(page_urls[i])
    soup = BeautifulSoup(page.content, 'html.parser')
    for link in soup.find_all('a'):
        time.sleep(0.5)
        pbar.update(1)
        pbar.set_description(f'Downloading page {i+1}/{len(page_urls)}', refresh=True)

        if link.has_attr('href') and link['href'].startswith('/talks/'):
            ted_url = 'https://www.ted.com'
            #check if link already exists in talks list
            if ted_url + link['href'] not in talks:
                #concat 'https://www.ted.com' + link['href'] to talks list
                talks.append(ted_url + link['href'])
            else:
        # if link already exists in talks list, skip it
                continue
            time.sleep(0.5)
pbar.close()

  0%|          | 0/1 [00:00<?, ?it/s]

In [6]:
talks[5]

'https://www.ted.com/talks/conor_russomanno_a_powerful_new_neurotech_tool_for_augmenting_your_mind'

In [13]:
import json

In [18]:

# find views of talk under <div class="text-sm w-full truncate text-gray-900" data-testid="talk-meta">
response = requests.get(talks[5])
soup = BeautifulSoup(response.text, 'lxml')
schema = soup.find('head').find('script', type='application/ld+json').text
schema = json.loads(schema)
schema.get('transcript')


'I became obsessed with the relationship between the brain and the mind after suffering a series of concussions playing football and rugby in college. I felt my mind change for years after. I was studying computers at the time, and it felt as though I had damaged my hardware and that my software was running differently. Over the following years, a close friend suffered a serious neck injury and multiple friends and family members were struggling with crippling mental health issues. All around me, people that I loved dearly were being afflicted by ailments of the nervous system or the mind. I was grappling with all of this while pursuing an MFA in Design and Technology at Parsons when a friend and fellow student showed me an open-source tutorial on how to build a low-cost single-channel EEG system to detect brain activity. After a couple long nights of hacking and tinkering, I saw my brainwaves dancing across the screen for the very first time. And that moment changed my life. In that m

In [150]:
pbar = tqdm(total=len(talks), dynamic_ncols=True, colour= 'green')

for i, ad in enumerate(talks):
    #-------create dataframe--------#
    df = pd.DataFrame(columns=["author", "talk", "description", "likes", "views"])

    time.sleep(2)
    pbar.update(1)
    response = requests.get(talks[i])
    soup = BeautifulSoup(response.text, 'lxml')


    #--------Title Schema------------#
    title_schema = soup.find('head').find('title').text.strip()

    #--------Description Schema------------#
    try:
        description_schema = soup.find('head').find('meta', attrs={'name':'description'})['content'].strip()

    except:
        description_schema = ''

    #--------Likes Schema------------#
    likes_schema = soup.find_all('span')[0].get_text().strip()

    #--------Views Schema------------#
    views_schema = soup.find_all('div', class_='text-sm w-full truncate text-gray-900')


    # get author name from title 
    author = title_schema.split(':')[0]
    talk = title_schema.split(':')[1].strip().replace('| TED Talk', '')
    description = description_schema
    likes = likes_schema.replace('(', '').replace(')', '')
    try: 
        views = views_schema[0].get_text().strip().split()[0]
    except:
        views = 0

    pbar.set_description(f'Downloading {talk}', refresh=True)

    # add to dataframe
    df = df.append({'author': author, 'talk': talk, 'description': description, 'likes': likes, 'views': views}, ignore_index=True)

 # ----------------------------------------Saving to Database--------------------------------------------#
    import sqlite3
    conn = sqlite3.connect('talks.db')
    cur = conn.cursor()
                
    for i in range(len(df)):
        cur.execute("INSERT INTO talks (author, talk, description, likes, views) VALUES (?, ?, ?, ?, ?)", (df.iloc[i]['author'], df.iloc[i]['talk'], df.iloc[i]['description'], df.iloc[i]['likes'], df.iloc[i]['views']))
    conn.commit()

    time.sleep(1)
    pbar.set_description(f'Adding {talk} to database', refresh=True)
    pbar.update(1)

    time.sleep(1)
    cur.execute("SELECT * FROM talks")
    rows = cur.fetchall()

    pbar.set_description(f"There are {len(rows)} records in the database", refresh=True)
    conn.close()
    time.sleep(1)
pbar.close()



/var/folders/7f/4z7lvktj44g121hm_1s6v18h0000gn/T/ipykernel_26163/4013203375.py:39: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'author': author, 'talk': talk, 'description': description, 'likes': likes, 'views': views}, ignore_index=True)
/var/folders/7f/4z7lvktj44g121hm_1s6v18h0000gn/T/ipykernel_26163/4013203375.py:39: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'author': author, 'talk': talk, 'description': description, 'likes': likes, 'views': views}, ignore_index=True)
/var/folders/7f/4z7lvktj44g121hm_1s6v18h0000gn/T/ipykernel_26163/4013203375.py:39: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'author': author, 'talk': talk, 'description': description, 'likes': likes

KeyboardInterrupt: 